# Utviklingsstatistikk

[Oppsummering](#oppsummering)

[Formål](#formål)

[Utfordring med å måle sikker og stabil drift](#utfordring)

[Måleparametere](#måleparametere)

[Om systemet](#omsystemet)

[Målinger](#målinger)

* [Kodestørrelse](#kodestørrelse)
* [Kodeendringer](#kodeendringer)
* [Netto vekst i kodelinjer](#nettovekst)
* [Detaljer i netto vekst i kodelinjer](#nettovekstdetaljer)
* [Oppdatering av avhengigheter](#oppdateringavhengigheter)
* [Løpende endringer i systemet](#løpendeendringer)
* [Risiko](#risiko)




# Oppsummering <a class="anchor" id="oppsummering"></a>

[TBW]

# Formål <a class="anchor" id="formål"></a>

Teamet har blitt bedt om å måle på noe som kan indikere om vi har sikker og stabil drift.

# Utfordring med å måle sikker og stabil drift  <a class="anchor" id="utfordring"></a>

Utfordringen med tradisjonell måling av sikker og stabil drift er at den gjerne måler på ting som oppetid eller rettetid. Dette er målinger som først slår ut etter at driften ikke lenger er sikker og/eller stabil, og det kan ta lang tid fra driften faktisk har blitt problematisk til målingene slår ut.

Vår erfaring er at risiko øker når systemer ikke holdes vedlike. Risiko påvirkes av manglende vedlikehold gjennom flere prosesser:
1)	Manglende oppdatering av avhengigheter gir økt sikkerhetsrisiko
2)	Manglende teknisk oppdatering gir økt sikkerhetsrisiko og risiko for lav kjennskap til utdaterte tekniske løsninger
3)	Lite andre endringer reduserer kunnskapen om systemet som øker rettetid og øker sjansen for feil når man en sjelden gang gjør enderinger

Vi mener det er verdt å forsøke å måle på utviklingsprosessen for å se om det kan gi indikatorer som sier noe om risikobildet.

Siden det er kostnader ved å gjøre målinger har vi sett etter målinger som er forholdsvis lette å innhente og som kan gi en indikasjon på at systemet vedlikeholdes.


# Måleparametere  <a class="anchor" id="måleparametere"></a>

Teamet tok sommeren 2022 i bruk teknikkene micro-commits og Arlo’s commit notation – dette gjør det mulig å benytte commit-statistikk til å si noe om omfang, intensjon og risiko ved commit’er.

Teamet har besluttet å se om
•	tellinger og forholdstall på ulike typer commits og
•	måling på kodestørrelse
kan indikere hvordan utviklingen er – og predikere eller illustrere en situasjon der det vil oppstå økt risiko for ustabil / usikker drift.

In [ ]:
# Setter ROOT_DIR
import os, sys
ROOT_DIR = os.path.dirname(os.path.abspath("")) if os.path.basename(os.path.abspath("")) != "eessi-pensjon" else os.path.abspath("")
os.chdir(ROOT_DIR)
sys.path.append(f'${ROOT_DIR}/ep-meta-analyse')

In [ ]:
# Henter inn commit-data
from datetime import datetime
from lib.commit_data import gather_changes_from_subprojects

startdate = datetime(2022, 1, 1)
commits_dict = gather_changes_from_subprojects(ROOT_DIR, datetime.now() - startdate)

In [ ]:
# Konverter data til Pandas dataframe
import pandas as pd
import numpy as np

df = pd.DataFrame(commits_dict)

# Fiks datoformat fra python til pandas (og samme tidssone)
df['timestamp'] = df['timestamp'].apply(lambda x: pd.to_datetime(x).tz_convert('Europe/Berlin'))
df['risk'] = pd.to_numeric(df['risk'])
df['commit'] = df['hash']

# Legg på måned og tertial

df['Måned'] = pd.PeriodIndex(df['timestamp'], freq='M')
df['Tertial'] = df["timestamp"].apply(lambda x: f"{x.year}T{int((x.month - 1) / 4) + 1}")
# Legger til noen kjekke kolonner
df['BDFRT'] = np.where(df['intention'].isin(['B', 'D', 'F', 'R', 'T']), True, False)
df['UE'] = np.where(df['intention'].isin(['U', 'E']), True, False)

# Mer lesbar intention
conditions = [df['intention'] == 'E', df['intention'] == 'A', df['intention'] == 'P',
              df['intention'] == 'U', df['intention'] == 'F', df['intention'] == 'R',
              df['intention'] == 'D', df['intention'] == 'T', df['intention'] == 'B',
              df['intention'] == '*']
choices = ['E Miljø', 'A Automatisert', 'P Release (bibliotek)', 'U Avhengighet',
           'F Funksjonalitet', 'R Refaktorering', 'D Dokumentasjon', 'T Test',
           'B Feilretting', '* Uklassifisert']
df['Intention'] = np.select(conditions, choices, np.nan)

# Mer lesbar risk
conditions = [df['risk'] == 0, df['risk'] == 1, df['risk'] == 2, df['risk'] == 3,
              df['risk'] == 4, df['risk'] == 5, pd.isnull(df['risk'])]
choices = ['0 Ingen risiko', '1 Kjent trygg', '2 Validert', '3 Kjent restrisiko',
           '4 Feil/sannsynlig feil', '5 Ukjent risiko', np.nan]
df['Risk'] = np.select(conditions, choices, default=np.nan)

df['Kjent intention/risiko'] = np.where(df['intention'] != '*', True, False)

df_commits = df

In [ ]:
# Henter inn størrelsedata
from lib.size_data import size_data, change_data

size_dict = size_data(ROOT_DIR)

In [ ]:
df = pd.DataFrame(size_dict)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
for col in ['files', 'blank lines', 'comment lines', 'code lines']:
    df[col] = df[col].astype(int)

df_size = df

In [ ]:
 # Henter inn endring i størrelse-data

siste_tertial = "2023T1"
tertialer = ["2022T3", siste_tertial]

df_changes= {}
for tertial in tertialer:
    changes_dict = change_data(ROOT_DIR, tertial)

    df = pd.DataFrame(changes_dict)

    # Konverter "tall" til tall
    for col in df.columns[df.columns.get_loc('== files'):]:
        df[col] = df[col].astype(int)

    df_changes[tertial] = df

In [ ]:
df = df_commits[['Måned', 'Kjent intention/risiko']]

df = df.groupby(df['Måned']).mean()
fig = df.plot(kind='bar')

Vi observerer at [TBW]

# Om systemet <a class="anchor" id="omsystemet"></a>

Systemet består av en frontendapplikasjon (saksbehandling-ui), 10 backendapplikasjoner, 7 delte bibliotek på backend og et antall bibliotek på frontend (som foreløpig ikke er med i oversikten og statistikken).

|Applikasjoner||
|---|---|
|saksbehandling-ui|Frontend for saksbehandlere|
|oppgave|Oppgaver til Oppgave (asynkron)|
|krav-initialisering|Krav til Pesys (asynkron)|
|statistikk|Statistikk til Statistikk-folket (asynkron)|
|onprem-proxy|For å kalle fra GCP til/fra onprem (Pesys mm) (online)|
|saksbehandling-api|Fasade for diverse API for saksbehandling-ui (online)|
|begrens-innsyn|Merking av saker med beskyttelse (asynkron)|
|journalforing|Journalføring av dokumenter (asynkron)|
|prefill|Preutfylling av SED (online)|
|fagmodul|Diverse API for saksbehandling-ui m fl (online)|
|pdl-produsent|Oppdatering av PDL med id’er og adresser (asynkron)|

| Bibliotek              ||
|------------------------|---|
| ep-metrics             |Metrikk-bibliotek|
| ep-eux                 |Domene-modell SED/BUC mm|
| ep-logging             |Loggingsbibliotek|
| ep-personoppslag       |Klient for oppslag mot PDL|
| ep-security-sts        |Brukes onprem for token-utveksling|
| ep-pensjonsinformasjon |Klient for oppslag mot Pesys|
| ep-kodeverk            |Klient mm - opplag i kodeverk|

| Meta                   ||
| ---                    |---|
| Meta                   |Verktøy for å jobbe på tvers av modulene nevnt ovenfor|



# Målinger <a class="anchor" id="målinger"></a>

## Kodestørrelse <a class="anchor" id="kodestørrelse"></a>

Kodestørrelse i applikasjoner korrelerer godt med kompleksitet, som igjen korrelerer med feil- og vedlikeholdsomfang.

Vi teller kodelinjer med cloc[^1]. (Det er noen små frontend-bibliotek som deles mellom EESSI NAV og EESSI Pensjon som ikke er med i statistikken).

### Kodestørrelse

In [ ]:
print(f"Kodestørrelse pr {df_size['date'].dt.strftime('%d.%m.%Y')[0]}.")

In [ ]:
df = df_size[['language', 'files', 'code lines', 'comment lines']]
df.groupby('language').sum().sort_values(by='code lines', ascending=False)

In [ ]:
df = df_size[['language', 'code lines']]
df = df.groupby('language').sum().sort_values('code lines', ascending=False)

def autopct(pct): # only show the label when it's > 2%
    return ('%.0f' % pct + '%') if pct > 2 else ''

plt = df.plot.pie(y='code lines', autopct=autopct, labeldistance=None).legend(loc='center left',bbox_to_anchor=(1.0, 0.5))

Koden er stort sett skrevet i Kotlin og TypeScript, og noe LESS– øvrig kode er knyttet til bygg, deploy, konfigurasjon og diverse utviklings- og støtteverktøy.


[^1]: Mer om cloc-kommandoen:
```
cloc --csv --vcs git --exclude-dir=dist,build,gradle --exclude-list-file=.clocignore --exclude-lang=JSON,XML --quiet .
```
der .clocignore inneholder:
```
gradlew
gradlew.bat
public/pdf.worker.js
public/static/js/pdf.worker.js
package-lock.json
src/components/PostalCodes/Postal-codes-Norway-ansi.ts
src/minibootstrap.less
```

In [ ]:
df = df_size[['module', 'code lines']]
df = df.groupby('module').sum().sort_values(by='code lines', ascending=False)
df['code lines %'] = df['code lines'] / df['code lines'].sum() * 100
df

Frontend-app’en for saksbehandling utgjør om lag en tredel av den totale koden, i én app.

In [ ]:
plot = df.plot.pie(y='code lines', legend=False, ylabel='', title="code by module")

In [ ]:
df = df_size[['module', 'language', 'code lines']]
df = df.groupby(['module', 'language']).sum().unstack()

plt = df.plot(kind='bar', stacked=True, title="modules and languages").legend(loc='center left',bbox_to_anchor=(1.0, 0.5))


## Kodeendringer <a class="anchor" id="kodeendringer"></a>

Vi kan se på hvilke app’er det er gjort kodeendringer i ved å sammenlikne koden på starten og slutten av tertialet.

In [ ]:

df = df_changes[siste_tertial][['type', 'module', '!= code', '+ code', '- code']].copy()

df['touched code'] = df['!= code'] + df['+ code'] + (df['- code'] / 2).astype(int)

df = df[['type', 'module', 'touched code']]

plt = df.groupby(['type', 'module']).sum().sort_index(ascending=True).plot.barh(title='"Rørte" kodelinjer (endrede + nye + slettede/2)')

Vi observerer at [TBW]

### Hva betyr dette?

[TBW]

## Netto vekst i kodelinjer <a class="anchor" id="nettovekst"></a>

Netto vekst i kodelinjer indikerer at det blir mer kode å vedlikeholde. Refaktorering kan til en viss grad motvirke veksten.

In [ ]:
df = df_changes[siste_tertial].loc[df_changes[siste_tertial]['language'].isin(['Kotlin', 'TypeScript'])].copy()

df['net code lines'] = df['+ code'] - df['- code']
df = df[['type', 'module', 'net code lines']]

plt = df.groupby(['type', 'module']).sum().sort_index(ascending=True).plot.barh(title='Netto vekst i kodelinjer (Kotlin og Typescript)')

### Hva betyr dette?

[TBW]

## Detaljer i netto vekst i kodelinjer <a class="anchor" id="nettovekstdetaljer"></a>

In [ ]:
df = df_changes[siste_tertial][['module', 'language', '+ code', '- code']].copy()
df['net code lines'] = df['+ code'] - df['- code']
pivot = df.pivot(index='module', columns='language', values='net code lines')
pivot = pivot.fillna(0)
pivot.loc[:, 'Total']= pivot.sum(axis=1)
pivot = pivot.sort_values(['Total'], ascending=False)
pivot.loc['Total',:]= pivot.sum(axis=0)
pivot = pivot.astype(int)
pivot

Vi observerer at [TBW]

### Hva betyr dette?

[TBW]

## Oppdatering av avhengigheter <a class="anchor" id="oppdateringavhengigheter"></a>

Oppdateringer av avhengigheter reduserer risiko for sikkerhetshull, og er med på å redusere problemene med sporadiske oppdateringer (mange ting som oppdateres samtidig, og at store hopp i versjoner gjør det vanskeligere å finne feil dersom noe går galt).

In [ ]:
df = df_commits.loc[df_commits['UE']][['Tertial', 'Intention', 'commit']].loc[df_commits['Tertial'].isin(tertialer)]

df = df.groupby(['Tertial', 'Intention']).count().unstack()
fig = df.plot(kind='bar', stacked=True, title="UE-commits pr tertial")

Vi observerer at [TBW]

#### Oppdateringer av avhengigheter pr modul og måned

In [ ]:
df = df_commits.loc[df_commits['Tertial'] == siste_tertial].loc[df_commits['intention'] == 'U'][['Måned', 'type', 'module', 'commit']]

pivot = df.groupby(['type', 'module', 'Måned']).count().unstack().fillna(0).astype(int)

pivot.loc[:, 'Total'] = pivot.sum(axis=1)
pivot = pivot.sort_values(['type','Total'], ascending=[True,False])
pivot

### Hva betyr dette?

[TBW]

## Løpende endringer i systemet <a class="anchor" id="løpendeendringer"></a>

At det gjøres endringer i systemet er indikator på at systemet vedlikeholdes, noe som opprettholder kjennskapen til systemet. Vi ser her på endringer merket med B, D, F, R og T; de er knyttet til feilretting (B), dokumentasjon (nær/i koden) (D), funksjonalitet (F), interne forbedringer/refaktorering (R) og testforbedringer (T).

In [ ]:
df = df_commits.loc[df_commits['BDFRT']][['Tertial', 'Intention', 'commit']].loc[df_commits['Tertial'].isin(tertialer)]

df = df.groupby(['Tertial', 'Intention']).count().unstack()
fig = df.plot(kind='bar', stacked=True, title="BDFRT-commits pr tertial").legend(loc='center left',bbox_to_anchor=(1.0, 0.5))

In [ ]:
df = df_commits.loc[df_commits['intention'].isin(['B','F','R'])][['Tertial', 'Intention', 'commit']].loc[df_commits['Tertial'].isin(tertialer)]

df = df.groupby(['Tertial', 'Intention']).count().unstack()

df_total = df.sum(axis=1)
df_rel = df.div(df_total, 0) * 100

fig = df_rel.plot(kind='bar', stacked=True, title="BFR-commits %-vis fordeling").legend(loc='center left',bbox_to_anchor=(1.0, 0.5))

Vi observerer at [TBW]

In [ ]:
df = df_commits.loc[df_commits['BDFRT']].loc[df_commits['Tertial'] == siste_tertial].loc[df_commits['type']=='app'][['module', 'commit']]

df = df.groupby('module').count().sort_values(by='commit', ascending=False)

def autopct(pct):
    return ('%.0f' % pct + '%') if pct > 3 else ''

plt = df.plot.pie(
    y='commit',
    title=f'Fordeling av BDFRT-commits på app-er (bibliotek er utelatt) i {tertial}',
    autopct=autopct, labeldistance=None).legend(loc='center left',bbox_to_anchor=(1.0, 0.5))


Vi observerer at [TBW]

#### BDFRT-oppdateringer pr modul

In [ ]:
df = df_commits.loc[df_commits['BDFRT']].loc[df_commits['Tertial'] == siste_tertial][['type', 'module', 'intention', 'commit']]

df = df.groupby(['type', 'module', 'intention']).count().unstack()

df['Total'] = df.sum(axis=1)

df = df.fillna(0).astype(int).sort_values(by=['type', 'Total'], ascending=[True, False])
df

Vi observerer at [TBW]

### Hva betyr dette?

[TBW]


## Risiko <a class="anchor" id="risiko"></a>

Vi merker commits med risiko, med en skala fra 0 til 5:

0)	Ingen risiko
1)	Kjent trygg
2)	Validert
3)	Kjent restrisiko
4)	Feil/sannsynlig feil
5)	Ukjent risiko

Endringer med risiko 1 Kjent trygg har svært lav risiko og krever lite eller ingen kjøring av tester. Dokumentasjons- og testendringer skal ha risiko 1. For endringer i produksjonskode er det strenge regler for hvilke endringer som kan merkes risiko 1 – stort sett går det ut på at det er små endringer som er gjort med støtte av funksjonalitet i utviklingsverktøyet (IDE).

Endringer med risiko 2 Validert krever som regel at tester kjøres lokalt for at man skal bli trygg, noe som kan ta litt tid, avhengig av app og oppsett. For oppdateringer av avhengigheter er det ofte vanskelig å si hva risiko er, men vi kjører tester for alle oppdateringer så vi merker dem rutinemessig med risiko 2.

Endringer med risiko 3 Kjent restrisiko må ofte følges opp litt ekstra i forbindelse med produksjonssetting, eller de må testes manuelt før de går i produksjon.

Risiko 4 er normalt ikke akseptabelt å dytte til produksjon – dette er endringer som er uferdige eller har høy risiko.

Risiko 5 betyr at risiko er ukjent. Ofte vet man mer i situasjonen, men kunnskapen om risiko er tapt i ettertid (må eventuelt vurderes på nytt).

Det er nok litt mer slurv med vurdering av risikonivå enn det er med vurdering av intensjon.

De fleste endringer i produksjonskode vil være risiko fra 1 til 3

In [ ]:
df = df_commits.loc[df_commits['intention'].isin(['B','F','R'])].loc[df_commits['Tertial'].isin(tertialer)][['Tertial', 'Risk', 'commit']]

df = df.groupby(['Tertial', 'Risk']).count().unstack()

df_total = df.sum(axis=1)
df_rel = df.div(df_total, 0)*100

fig = df_rel.plot(kind='bar', stacked=True, title="Risiko BFR-commits %-vis fordeling").legend(loc='center left',bbox_to_anchor=(1.0, 0.5))

Vi observerer at [TBW]

### Hva betyr dette?

[TBW]